In [1]:
import pandas as pd
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp

In [2]:
resultados = pd.read_csv('../validation_results.csv')
resultados

,Unnamed: 0,CLASSIFIER,EMBEDDER,PRECISION,RECALL,F1SCORE,TIME
0,0,KNN,TFIDF,0.586269,0.586269,0.586269,8.590708
1,1,KNN,PRETRAINED,0.576332,0.576332,0.576332,9.957644
2,2,KNN,TRAINED,0.580397,0.580397,0.580397,5.361411
3,3,SVM,TFIDF,0.116983,0.116983,0.116983,658.436923
4,4,SVM,PRETRAINED,0.121048,0.121048,0.121048,137.842464
...,...,...,...,...,...,...,...
85,85,SVM,PRETRAINED,0.111161,0.111161,0.111161,126.038186
86,86,SVM,TRAINED,0.523723,0.523723,0.523723,95.171705
87,87,MLP,TFIDF,0.812020,0.812020,0.812020,2531.164760
88,88,MLP,PRETRAINED,0.629914,0.629914,0.629914,208.423105


In [3]:
dados_por_clf_emb = []
for clf in resultados.CLASSIFIER.unique():
    for emb in resultados.EMBEDDER.unique():
        dados_por_clf_emb.append({clf+"_"+emb: resultados[(resultados.CLASSIFIER == clf) & (resultados.EMBEDDER == emb)].F1SCORE.tolist()})

In [4]:
dados_para_avaliar = [list(d.values())[0] for d in dados_por_clf_emb]

In [5]:
# src: https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/
#Fail to Reject H0: Paired sample distributions are equal.
#Reject H0: Paired sample distributions are not equal.
stat, p = friedmanchisquare(*dados_para_avaliar)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Same distributions (fail to reject H0)')
else:
	print('Different distributions (reject H0)')

Statistics=79.120, p=0.000
Different distributions (reject H0)


In [6]:
sp.posthoc_nemenyi_friedman(dados_para_avaliar)
#preciso entender como avaliar isso

,0,1,2,3,4,5,6,7,8,9
0,-1.000000,0.900000,0.900000,0.900000,0.570115,0.195798,0.273159,0.900000,0.900000,0.809443
1,0.900000,-1.000000,0.900000,0.900000,0.273159,0.058115,0.089875,0.761574,0.900000,0.522249
2,0.900000,0.900000,-1.000000,0.900000,0.641912,0.252106,0.341734,0.900000,0.900000,0.881240
3,0.900000,0.900000,0.900000,-1.000000,0.040788,0.004577,0.008272,0.295079,0.900000,0.123217
4,0.570115,0.273159,0.641912,0.040788,-1.000000,0.900000,0.900000,0.900000,0.367342,0.900000
5,0.195798,0.058115,0.252106,0.004577,0.900000,-1.000000,0.900000,0.900000,0.089875,0.900000
6,0.273159,0.089875,0.341734,0.008272,0.900000,0.900000,-1.000000,0.900000,0.134922,0.900000
7,0.900000,0.761574,0.900000,0.295079,0.900000,0.900000,0.900000,-1.000000,0.857305,0.900000
8,0.900000,0.900000,0.900000,0.900000,0.367342,0.089875,0.134922,0.857305,-1.000000,0.617980
9,0.809443,0.522249,0.881240,0.123217,0.900000,0.900000,0.900000,0.900000,0.617980,-1.000000
